## Day 85 Lecture 1 Assignment

In this assignment, we will learn how to use the other layers to improve our model performance.

In [1]:
import numpy as np
import pandas as pd

We will explore a dataset containing information about twitter users and will detect whether or not the user is a bot.

In [2]:
twitter = pd.read_csv('https://tf-assets-prod.s3.amazonaws.com/tf-curric/data-science/training_data_2_csv_UTF.csv')

In [3]:
twitter.head()

,id,id_str,screen_name,location,description,url,followers_count,friends_count,listed_count,created_at,favourites_count,verified,statuses_count,lang,status,default_profile,default_profile_image,has_extended_profile,name,bot
0,8.160000e+17,"""815745789754417152""","""HoustonPokeMap""","""Houston, TX""","""Rare and strong PokŽmon in Houston, TX. See m...","""https://t.co/dnWuDbFRkt""",1291,0,10,"""Mon Jan 02 02:25:26 +0000 2017""",0,False,78554,"""en""","{\r ""created_at"": ""Sun Mar 12 15:44:04 +0...",True,False,False,"""Houston PokŽ Alert""",1
1,4.843621e+09,4843621225,kernyeahx,"Templeville town, MD, USA",From late 2014 Socium Marketplace will make sh...,NaN,1,349,0,2/1/2016 7:37,38,False,31,en,NaN,True,False,False,Keri Nelson,1
2,4.303727e+09,4303727112,mattlieberisbot,NaN,"Inspired by the smart, funny folks at @replyal...",https://t.co/P1e1o0m4KC,1086,0,14,Fri Nov 20 18:53:22 +0000 2015,0,False,713,en,"{'retweeted': False, 'is_quote_status': False,...",True,False,False,Matt Lieber Is Bot,1
3,3.063139e+09,3063139353,sc_papers,NaN,NaN,NaN,33,0,8,2/25/2015 20:11,0,False,676,en,Construction of human anti-tetanus single-chai...,True,True,False,single cell papers,1
4,2.955142e+09,2955142070,lucarivera16,"Dublin, United States",Inspiring cooks everywhere since 1956.,NaN,11,745,0,1/1/2015 17:44,146,False,185,en,NaN,False,False,False,lucarivera16,1


In [4]:
twitter.shape

(2797, 20)

Start by getting rid of all columns that are not useful.

Next, get rid of all columns that contain more than 30% missing data. After that, remove all rows containing at least one missing observation.

In [5]:
#A summary of missing variables represented as a percentage of the total missing content. 
def missingness_summary(df, print_log=False, sort='ascending'):
  s = df.isnull().sum()*100/df.isnull().count()
    
  if sort.lower() == 'ascending':
    s = s.sort_values(ascending=True)
  elif sort.lower() == 'descending':
    s = s.sort_values(ascending=False)  
  if print_log: 
    print(s)
  
  return pd.Series(s)

In [6]:
# Answer below:
missingness_summary(twitter)

id                        0.000000
default_profile_image     0.000000
default_profile           0.000000
lang                      0.000000
statuses_count            0.000000
verified                  0.000000
favourites_count          0.000000
name                      0.000000
created_at                0.000000
friends_count             0.000000
followers_count           0.000000
screen_name               0.000000
id_str                    0.000000
listed_count              0.000000
bot                       0.000000
has_extended_profile      3.539507
status                   10.332499
description              14.408295
location                 36.467644
url                      47.979979
dtype: float64

In [7]:
twitter[['id', 'id_str']].nunique()

id        2393
id_str    2432
dtype: int64

In [8]:
# Answer below:
data = twitter.drop(columns=['id', 'id_str', 'url', 'location']).dropna()
data.head()

,screen_name,description,followers_count,friends_count,listed_count,created_at,favourites_count,verified,statuses_count,lang,status,default_profile,default_profile_image,has_extended_profile,name,bot
0,"""HoustonPokeMap""","""Rare and strong PokŽmon in Houston, TX. See m...",1291,0,10,"""Mon Jan 02 02:25:26 +0000 2017""",0,False,78554,"""en""","{\r ""created_at"": ""Sun Mar 12 15:44:04 +0...",True,False,False,"""Houston PokŽ Alert""",1
2,mattlieberisbot,"Inspired by the smart, funny folks at @replyal...",1086,0,14,Fri Nov 20 18:53:22 +0000 2015,0,False,713,en,"{'retweeted': False, 'is_quote_status': False,...",True,False,False,Matt Lieber Is Bot,1
5,dantheimprover,Just a guy trying to do good by telling everyo...,1,186,0,13/03/2017 22:53,0,False,11,en,Status(_api=<tweepy.api.API object at 0x101927...,True,False,True,dantheimprover,1
6,_all_of_us_,bot by @rubicon,193,0,19,Wed May 07 22:29:25 +0000 2014,0,False,6068,en,"{u'contributors': None, u'truncated': False, u...",False,False,False,everything always,1
7,KatamariItems,[Bot rolled up by @BeachEpisode] Cataloguing e...,8227,2,89,Thu Jun 18 22:07:31 +0000 2015,26,False,2597,en,"{u'contributors': None, u'truncated': False, u...",True,False,False,Katamari Collection,1


Now we will use our embedding functions from a previous assignment.

In [9]:
import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords
import re
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

def remove_stopwords(input_text):
        stopwords_list = stopwords.words('english')
        # Some words which might indicate a certain sentiment are kept via a whitelist
        whitelist = ["n't", "not", "no"]
        words = input_text.split() 
        clean_words = [word for word in words if (word not in stopwords_list or word in whitelist) and len(word) > 1] 
        return " ".join(clean_words)       

def stem_list(word_list):
    stemmed = []
    for word in word_list:
        stemmedword = stemmer.stem(word)
        stemmed.append(stemmedword)
    return stemmed

def normalize(terms):
    terms = terms.lower()
    terms = remove_stopwords(terms)
    word_delimiters = u'[\\[\\]\n.!?,;:\t\\-\\"\\(\\)\\\'\u2019\u2013 ]'
    term_list = re.split(word_delimiters, terms)
    trimmed = [x.rstrip() for x in term_list]
    stemmed = stem_list(trimmed)
    space = ' '
    normed = space.join(stemmed)
    normed = normed.replace('  ', ' ')
    return normed

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


We will create two branches, one branch will process the text data in the description and the other will process all other columns. First, create a numpy array with the encoded data from the description column. Normalize each description, one hot encode the text, pad the row and create a numpy array.

In [10]:
# Answer below:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import one_hot
data['desc_norm'] = data.description.apply(normalize)
enc = data.desc_norm.apply(one_hot, args=[5000])
enc = pad_sequences(enc)
enc

array([[   0,    0,    0, ...,  492, 1709, 1547],
       [   0,    0,    0, ..., 1311, 4636,  581],
       [   0,    0,    0, ..., 2575, 3313, 3888],
       ...,
       [   0,    0,    0, ...,    0,    0, 3623],
       [   0,    0,    0, ..., 4253, 1955, 2484],
       [   0,    0,    0, ..., 2541,  440,  448]], dtype=int32)

Convert all boolean variables to numeric (zero for false and 1 for true)

In [11]:
# Answer below:
bv = data.dtypes[data.dtypes.values == bool].keys()
for val in bv:
  data[val] = data[val].astype(int)

data.dtypes

screen_name              object
description              object
followers_count           int64
friends_count             int64
listed_count              int64
created_at               object
favourites_count          int64
verified                  int64
statuses_count            int64
lang                     object
status                   object
default_profile           int64
default_profile_image     int64
has_extended_profile     object
name                     object
bot                       int64
desc_norm                object
dtype: object

In [12]:
data.describe()

,followers_count,friends_count,listed_count,favourites_count,verified,statuses_count,default_profile,default_profile_image,bot
count,2.121000e+03,2121.000000,2121.000000,2121.000000,2121.000000,2.121000e+03,2121.000000,2121.000000,2121.000000
mean,1.163806e+06,4813.884960,3801.734088,2525.816124,0.287129,2.468691e+04,0.378124,0.015087,0.463932
std,5.943476e+06,45589.697276,18842.921887,18388.214376,0.452529,1.768575e+05,0.485033,0.121929,0.498815
min,0.000000e+00,0.000000,0.000000,0.000000,0.000000,1.000000e+00,0.000000,0.000000,0.000000
25%,1.130000e+02,3.000000,7.000000,0.000000,0.000000,6.340000e+02,0.000000,0.000000,0.000000
50%,9.040000e+02,183.000000,36.000000,42.000000,0.000000,3.918000e+03,0.000000,0.000000,0.000000
75%,4.884800e+04,763.000000,836.000000,770.000000,1.000000,1.393600e+04,1.000000,0.000000,1.000000
max,9.632156e+07,985943.000000,620331.000000,714021.000000,1.000000,6.863363e+06,1.000000,1.000000,1.000000


Create dummy variables out of the relevant object columns. Take caution when converting columns that may incorrectly classified as object.

In [13]:
data[['lang', 'has_extended_profile', ]].head()

,lang,has_extended_profile
0,"""en""",False
2,en,False
5,en,True
6,en,False
7,en,False


In [14]:
data[['lang', 'has_extended_profile', ]].nunique()

lang                    27
has_extended_profile     2
dtype: int64

In [15]:
data.has_extended_profile = data.has_extended_profile.astype(int)

In [16]:
data.has_extended_profile.value_counts()

0    1788
1     333
Name: has_extended_profile, dtype: int64

In [17]:
data.lang.value_counts()

en         1883
"en"         94
en-gb        52
es           19
pt           14
fr            8
de            8
it            6
"ja"          6
ja            4
"es"          3
"pt"          3
nl            2
ru            2
zh-CN         2
zh-cn         2
"en-gb"       2
tr            2
"zh-cn"       1
"zh-CN"       1
ta            1
"fr"          1
gl            1
"ko"          1
"da"          1
"it"          1
ar            1
Name: lang, dtype: int64

In [18]:
data.lang = data.lang.str.replace('"', '')

In [19]:
# Answer below:
data.lang.value_counts()


en       1977
en-gb      54
es         22
pt         17
ja         10
fr          9
de          8
it          7
zh-cn       3
zh-CN       3
ru          2
tr          2
nl          2
da          1
ta          1
ko          1
ar          1
gl          1
Name: lang, dtype: int64

In [20]:
data.screen_name.nunique()

2114

In [21]:
data = pd.get_dummies(data, drop_first=True, columns=['lang'])
data.head()

,screen_name,description,followers_count,friends_count,listed_count,created_at,favourites_count,verified,statuses_count,status,default_profile,default_profile_image,has_extended_profile,name,bot,desc_norm,lang_da,lang_de,lang_en,lang_en-gb,lang_es,lang_fr,lang_gl,lang_it,lang_ja,lang_ko,lang_nl,lang_pt,lang_ru,lang_ta,lang_tr,lang_zh-CN,lang_zh-cn
0,"""HoustonPokeMap""","""Rare and strong PokŽmon in Houston, TX. See m...",1291,0,10,"""Mon Jan 02 02:25:26 +0000 2017""",0,0,78554,"{\r ""created_at"": ""Sun Mar 12 15:44:04 +0...",1,0,0,"""Houston PokŽ Alert""",1,rare strong pokžmon houston tx see pokžmon ht...,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,mattlieberisbot,"Inspired by the smart, funny folks at @replyal...",1086,0,14,Fri Nov 20 18:53:22 +0000 2015,0,0,713,"{'retweeted': False, 'is_quote_status': False,...",1,0,0,Matt Lieber Is Bot,1,inspir smart funni folk @replyal @gimletmedia ...,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,dantheimprover,Just a guy trying to do good by telling everyo...,1,186,0,13/03/2017 22:53,0,0,11,Status(_api=<tweepy.api.API object at 0x101927...,1,0,1,dantheimprover,1,guy tri good tell everyon els get better,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,_all_of_us_,bot by @rubicon,193,0,19,Wed May 07 22:29:25 +0000 2014,0,0,6068,"{u'contributors': None, u'truncated': False, u...",0,0,0,everything always,1,bot @rubicon,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,KatamariItems,[Bot rolled up by @BeachEpisode] Cataloguing e...,8227,2,89,Thu Jun 18 22:07:31 +0000 2015,26,0,2597,"{u'contributors': None, u'truncated': False, u...",1,0,0,Katamari Collection,1,bot roll @beachepisod catalogu everi item kat...,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


Min max scale the data decribing each user (do not min max scale the word embeddings).

In [22]:
data.columns

Index(['screen_name', 'description', 'followers_count', 'friends_count',
       'listed_count', 'created_at', 'favourites_count', 'verified',
       'statuses_count', 'status', 'default_profile', 'default_profile_image',
       'has_extended_profile', 'name', 'bot', 'desc_norm', 'lang_da',
       'lang_de', 'lang_en', 'lang_en-gb', 'lang_es', 'lang_fr', 'lang_gl',
       'lang_it', 'lang_ja', 'lang_ko', 'lang_nl', 'lang_pt', 'lang_ru',
       'lang_ta', 'lang_tr', 'lang_zh-CN', 'lang_zh-cn'],
      dtype='object')

In [23]:
# Answer below:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
X_desc =data[['followers_count', 'friends_count',
       'listed_count', 'favourites_count', 'verified',
       'statuses_count', 'default_profile', 'default_profile_image',
       'has_extended_profile', 'lang_da',
       'lang_de', 'lang_en', 'lang_en-gb', 'lang_es', 'lang_fr', 'lang_gl',
       'lang_it', 'lang_ja', 'lang_ko', 'lang_nl', 'lang_pt', 'lang_ru',
       'lang_ta', 'lang_tr', 'lang_zh-CN', 'lang_zh-cn']]
X_desc = scaler.fit_transform(X_desc)
y = data.bot

Now we'll create the two branches. Create a model for the numeric data that consists of 3 dense layers. An input layer and two hidden layers of size 32.

In [24]:
X_desc.shape

(2121, 26)

In [25]:
#NUMERIC BRANCH  

# Answer below:
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model

# create input
inputs = Input(shape=X_desc.shape[1], name='numeric_input')

# hidden layers
h1 = Dense(32, activation='relu', name='numeric_hidden_1')(inputs)
h2 = Dense(32, activation='relu', name='numeric_hidden_2')(h1)

Create the second branch of the model using the encoded words. This branch will consist of 4 layers: An input layer, an embedding layer returning data of dimension 100, an LSTM layer of unit size 32 and a dense layer of unit size 32. 

In [26]:
#WORD BRANCH
from tensorflow.keras.layers import LSTM, Embedding, Input, Flatten, Dense

# input layer
inputTensor = Input(shape=(enc.shape[1], ), name='word_input' )

# embedding layer
embed = Embedding(5000, 100, input_length=(enc.shape[1], ), name='word_embed')(inputTensor)


# lstm layer
lstm = LSTM(32, name='word_lstm')(embed)

# dense layer
h1 = Dense(32, activation='relu', name='word_hidden_1')(lstm)


Merge the two models using the `concatenate` function (merge the two final dense layers in each branch) and create an output dense layer.

In [27]:
from tensorflow.keras.layers import concatenate
# Answer below:
concat = concatenate([h2, h1])

# output
output = Dense(1, activation='sigmoid', name='output')(concat) 

Create a model using the two inputs and the single output and print the summary

In [28]:
# Answer below: 
model = Model(inputs=[inputs, inputTensor], outputs=output)

In [29]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
word_input (InputLayer)         [(None, 66)]         0                                            
__________________________________________________________________________________________________
numeric_input (InputLayer)      [(None, 26)]         0                                            
__________________________________________________________________________________________________
word_embed (Embedding)          (None, 66, 100)      500000      word_input[0][0]                 
__________________________________________________________________________________________________
numeric_hidden_1 (Dense)        (None, 32)           864         numeric_input[0][0]              
______________________________________________________________________________________________

Compile and fit the model using the appropriate optimizer, loss, and metrics. Train the model for 10 epochs with a batch size of 128.

In [30]:
# Answer below:
model.compile(optimizer='RMSProp', 
              loss='binary_crossentropy', 
              metrics=['accuracy']) 
    

In [31]:
model.fit(x=[X_desc, enc], y=y, epochs=10, batch_size=128)

Epoch 1/10
17/17 [==============================] - 3s 59ms/step - loss: 0.6734 - accuracy: 0.5653
Epoch 2/10
17/17 [==============================] - 1s 59ms/step - loss: 0.5431 - accuracy: 0.8741
Epoch 3/10
17/17 [==============================] - 1s 58ms/step - loss: 0.3243 - accuracy: 0.9133
Epoch 4/10
17/17 [==============================] - 1s 58ms/step - loss: 0.2264 - accuracy: 0.9313
Epoch 5/10
17/17 [==============================] - 1s 58ms/step - loss: 0.1539 - accuracy: 0.9492
Epoch 6/10
17/17 [==============================] - 1s 57ms/step - loss: 0.1135 - accuracy: 0.9724
Epoch 7/10
17/17 [==============================] - 1s 59ms/step - loss: 0.0914 - accuracy: 0.9679
Epoch 8/10
17/17 [==============================] - 1s 57ms/step - loss: 0.0538 - accuracy: 0.9864
Epoch 9/10
17/17 [==============================] - 1s 59ms/step - loss: 0.0416 - accuracy: 0.9899
Epoch 10/10
17/17 [==============================] - 1s 59ms/step - loss: 0.0410 - accuracy: 0.9879


In [32]:
#NUMERIC BRANCH  

# Answer below:
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model

# create input
inputs = Input(shape=X_desc.shape[1], name='numeric_input')

# hidden layers
h1 = Dense(32, activation='relu', name='numeric_hidden_1')(inputs)
h2 = Dense(32, activation='relu', name='numeric_hidden_2')(h1)


#WORD BRANCH
from tensorflow.keras.layers import LSTM, Embedding, Input, Flatten, Dense

# input layer
inputTensor = Input(shape=(enc.shape[1], ), name='word_input' )

# embedding layer
embed = Embedding(5000, 100, input_length=(enc.shape[1], ), name='word_embed')(inputTensor)


# lstm layer
lstm = LSTM(32, name='word_lstm')(embed)

# dense layer
h1 = Dense(32, activation='relu', name='word_hidden_1')(lstm)


from tensorflow.keras.layers import concatenate
# Answer below:
concat = concatenate([h2, h1])

# output
output = Dense(1, activation='sigmoid', name='output')(concat) 


# Answer below: 
model = Model(inputs=[inputs, inputTensor], outputs=output)

In [33]:
# Answer below:
model.compile(optimizer='adam', 
              loss='binary_crossentropy', 
              metrics=['accuracy']) 
    
model.fit(x=[X_desc, enc], y=y, epochs=10, batch_size=128)

Epoch 1/10
17/17 [==============================] - 3s 64ms/step - loss: 0.6750 - accuracy: 0.6779
Epoch 2/10
17/17 [==============================] - 1s 64ms/step - loss: 0.6150 - accuracy: 0.7488
Epoch 3/10
17/17 [==============================] - 1s 64ms/step - loss: 0.4365 - accuracy: 0.8985
Epoch 4/10
17/17 [==============================] - 1s 65ms/step - loss: 0.2296 - accuracy: 0.9478
Epoch 5/10
17/17 [==============================] - 1s 64ms/step - loss: 0.1322 - accuracy: 0.9638
Epoch 6/10
17/17 [==============================] - 1s 62ms/step - loss: 0.0878 - accuracy: 0.9812
Epoch 7/10
17/17 [==============================] - 1s 60ms/step - loss: 0.0507 - accuracy: 0.9871
Epoch 8/10
17/17 [==============================] - 1s 59ms/step - loss: 0.0379 - accuracy: 0.9915
Epoch 9/10
17/17 [==============================] - 1s 60ms/step - loss: 0.0414 - accuracy: 0.9920
Epoch 10/10
17/17 [==============================] - 1s 60ms/step - loss: 0.0296 - accuracy: 0.9886
